<a href="https://colab.research.google.com/github/danielfrgs/2023_1-Lenguaje_Natural/blob/main/Tareas/T2/A4/T2_A4_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import pandas as pd
import re
import numpy as np

# Tema 2 | Actividad 4

Fragoso Alvarado Daniel

In [8]:
# Vamos a abrir y leer los datos de forma que cada renglón sea un registro de una 
# lista, para poder iterar despues.
with open('/content/drive/MyDrive/Universidad/Ciencia de Datos/7mo Semestre/2023_1-Lenguaje_Natural/Tareas/T2/A4/dates.txt') as f:
    contents = f.readlines()

In [12]:
print(f'En el Dataset hay en total: {len(contents)} renglones, por lo que se espera la extracción de {len(contents)} fechas.')

En el Dataset hay en total: 500 renglones, por lo que se espera la extracción de 500 fechas.


Ahora vamos a generar una expresión regular para cada formato de fecha que encontremos. Cada expresión regular vamos a almacenarla en una cadena, que usaremos déspues para aplicarlas. 

1. De la forma $xx/xx/xx$, $xx/xx/xxxx$ & $x/x/xx$ 

Para los primeros digitos que hacen referencia a un mes pueden empezar con cero, o ser simplemente números entre el 1 y el 9, por ello agregamos "0?[0-9]", tambien puede ser el caso de que sea un número del 10 al 12, por lo que agregamos una opción "[12][0-9]"

También puede estar separada por una diagonal o un guión. por lo que agregamos "(/|-)"

Luego, seleccionamos el día, puede ser solo un número entre el 1-9, y puede o no empezar con 0: "0?[1-9]", luego puede ser un número entre 10 y 29: "[1-2][0-9]", y finalemnte puede darse el casod e ser un valor entre 30 y 31, y especificamos para evitar errores: "3[0-2]". Volvemos a agregar el separador: "(/|-)".

La parte del año puede ser un año completo de 4 digitos, o solo dos digitos, en caso de ser un año completo debe de empezar con 18, 19 o 20, y seguirle dos digitos, o solo dos digitos: "(18|19|20)\d\d|\d\d"

In [17]:
regex1 = r"(0?[0-9]|[12][0-9])(/|-)(0?[1-9]|[1-2][0-9]|3[0-2])(/|-)((18|19|20)\d\d|\d\d)"

2. De la forma $Mar-20-2009$; $Mar$ $20$, $2009$; $March$ $20$, $2009$; $Mar.$ $20$, $2009$; $Mar$ $20$ $2009$;

Primero, vamos a hacer la expresión para el mes; puede ser cualquier mes entre enero y diciembre, y puede o no tener abreviación (según el nombre en inglés), por lo que agregamos: "?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:enver)?|Dec(?:ember)?"

Para la separación, esta puede darse por un guión, un punto o un espacio, por lo que agregamos: "(?:|.?)( |-)"

El día puede ser un numero entre el 0-31, con o no inicio en cero: "[0-9][0-9]|0?[0-9]".

La separación puede ser un espacio, una coma, o un guión: "(?: |,|-?)(?: | ?)"

Y finalmente el año puede ser un año completo de 4 digitos, o solo dos digitos, en caso de ser un año completo debe de empezar con 18, 19 o 20, y seguirle dos digitos, o solo dos digitos: "(18|19|20)\d\d|\d\d".

In [18]:
regex2 = r"(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:enver)?|Dec(?:ember)?)(?:|.?)( |-)([0-9][0-9]|0?[0-9])(?: |,|-?)(?: | ?)(((18|19|20)\d\d))"

3. De la forma $12$ $March$ $2004$; $21$ $Oct$ $1977$

La estructura de esta es igual a la anterior solo en orden diferente:


In [19]:
regex3 = r"([0-9][0-9]|0?[0-9])(?:|.?)( |-)(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:enver)?|Dec(?:ember)?)(?: |,|-?)(?: | ?)(((18|19|20)\d\d))"

4. De la forma $Feb$ $2009$; $Sep$ $2009$; $Oct$ $2010$

Volvemos a copiar la esriutura anterior de la expresióm solo que eliminando la parte del día:

In [14]:
regex4 = r"(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:enver)?|Dec(?:ember))(?: |,|-?)(?: | ?)(((18|19|20)\d\d))"

5. De la forma $6/2008$; $12/2009$

Vamos a copiar la expresión del primer formato omitiendo la parte del día:

In [20]:
regex5 = r"([0-9][0-9]|0?[0-9])/((18|19|20)\d\d|\d\d)"

6. De la forma $2009$; $2010$.

Vamos a copiar la expresión para el año de la expresión anterior:

In [15]:
regex6 = r"((18|19|20)\d\d)"

## Aplicación

Primero vamos a generar una lista con las expresioes regulares creadas, para poder iterar sobre estas:

In [21]:
# Generamos una lista con las expresiones regulares
regex = [regex1, regex2, regex3, regex4, regex5, regex6]

In [26]:
# Vamos a generar una lista vacia para poder agregar las fechas extraidas a esta:
fechas = []

# Vamos a iterar sobre la lista de resgitros
for i in contents:
  j = 0
  k = 0
  lis = []
  # Vamos a correr un ciclo while hasta que se haga match con alguna fecha
  while k != 1:
      matches = re.finditer(regex[j], i)
      # Al hacer match vamos a obtener el resgistro y concatenarlo a 
      # una lista
      for matchNum, match in enumerate(matches, start=1):
          lis = [match.group()]
      k = len(lis)
      j += 1
      # En caso de que no se haga match despues de las 6 expresiones, salimos.
      if j == 6:
          k = 1
  fechas = fechas + lis

In [27]:
print(f'La cantidad de fechas extraidas es: {len(fechas)}')

La cantidad de fechas extraidas es: 500


In [28]:
# Veamos como se ven:

fechas[:10]

['03/25/93',
 '6/18/85',
 '7/8/71',
 '9/27/75',
 '2/6/96',
 '7/06/79',
 '5/18/78',
 '10/24/89',
 '3/7/86',
 '4/10/71']

In [34]:
# Vamos a transformar a fechas

# La función ya asume que si falta un día toma el primero del mes,. y si flata un mes toma el primer dia del primer mes del año.
fechas = pd.to_datetime(fechas)

En la transformación debido a que no se especifica los valores de los años o meses, la frunción crea datos incosnsitentes, enctonces, vamos a buscar aquellos años que sean mayores al año 2022 (actual) y vamos a restarle 100 años:

In [31]:
dates = np.where(fechas.year > 2022, fechas - pd.offsets.DateOffset(years=100), fechas)

DateOffset permite restar 100 años a un dato que ya es del tipo datetime:

In [33]:
# Finalmmente Ordenammos
dates = sorted(dates)
dates

[numpy.datetime64('1971-04-10T00:00:00.000000000'),
 numpy.datetime64('1971-05-18T00:00:00.000000000'),
 numpy.datetime64('1971-07-08T00:00:00.000000000'),
 numpy.datetime64('1971-07-11T00:00:00.000000000'),
 numpy.datetime64('1971-09-12T00:00:00.000000000'),
 numpy.datetime64('1972-01-01T00:00:00.000000000'),
 numpy.datetime64('1972-01-13T00:00:00.000000000'),
 numpy.datetime64('1972-01-26T00:00:00.000000000'),
 numpy.datetime64('1972-05-06T00:00:00.000000000'),
 numpy.datetime64('1972-05-13T00:00:00.000000000'),
 numpy.datetime64('1972-06-10T00:00:00.000000000'),
 numpy.datetime64('1972-06-15T00:00:00.000000000'),
 numpy.datetime64('1972-07-20T00:00:00.000000000'),
 numpy.datetime64('1972-10-04T00:00:00.000000000'),
 numpy.datetime64('1972-11-30T00:00:00.000000000'),
 numpy.datetime64('1973-01-01T00:00:00.000000000'),
 numpy.datetime64('1973-02-01T00:00:00.000000000'),
 numpy.datetime64('1973-02-01T00:00:00.000000000'),
 numpy.datetime64('1973-02-14T00:00:00.000000000'),
 numpy.datet